In [22]:
import pandas as pd
import imageio
import os

def game_jeon(video_path, csv_path, output_folder, f_frame_count=92, l_frame_count=0):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c değişkeni kaydedilen pozisyon sayısını tutar
    c = 0
    
    # Dizin kontrolü ve oluşturma
    os.makedirs(output_folder, exist_ok=True)

    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'BLUE_ADD_GAME_JEON') | (df['matchLogItemType'] == 'RED_ADD_GAME_JEON')].iterrows():
        game_jeon_system_time = row['systemTime']

        # İlk önceki TIMEOUT değerini bul
        timeout_time = None
        for i in range(index - 1, -1, -1):
            if df.loc[i, 'matchLogItemType'] == 'TIMEOUT':
                timeout_time = df.loc[i, 'systemTime']
                break

        if timeout_time is None:
            print(f"Uyarı: {index}. indisteki GAME_JEON için TIMEOUT bulunamadı. Geçiliyor.")
            continue

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        timeout_video_time_seconds = (timeout_time - start_system_time) / 1000.0
        game_jeon_video_time_seconds = (game_jeon_system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        start_frame_number = int(timeout_video_time_seconds * fps)
        game_jeon_frame_number = int(game_jeon_video_time_seconds * fps)
        
        start_frame_number = max(0, min(frame_count - 1, start_frame_number))  # Frame sayısını kontrol etme
        game_jeon_frame_number = max(0, min(frame_count - 1, game_jeon_frame_number))  # Frame sayısını kontrol etme

        # Belirtilen sayıda önceki ve sonraki frame'leri al
        start_frame = max(0, start_frame_number - f_frame_count)
        end_frame = min(frame_count - 1, game_jeon_frame_number + l_frame_count)
        c += 1

        # Videoyu oku ve yeni videoya yaz
        output_video = []
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c -= 1
            continue

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'game_jeon_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    
    print(f'Yan hakemlerin verdikleri Game-Jeonlar {output_folder} dosyasına kaydedildi. Toplam {c} pozisyon.')



In [23]:
# Değişkenlerin tanımlanması
date = '02_13'
court_num = 3
cam_num = 'live'

video_konum = '/home/muhlabws4/AntalyaKodlari/downloads'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 13 FEBRUARY 2024 - 3 COURT.mp4'
video_path = f'{video_konum}/{video_name}'

csv_konum = '/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/matchlog'
csv_name = '02_13-320-334.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'




In [24]:
# Fonksiyon çağrısı
game_jeon(video_path, csv_path, f'{output_path}_game_jeon', f_frame_count=164, l_frame_count=20)

Hata: 813525. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.
Yan hakemlerin verdikleri Game-Jeonlar /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_game_jeon dosyasına kaydedildi. Toplam 52 pozisyon.


In [25]:
import pandas as pd
import imageio
import os
import imageio_ffmpeg as ffmpeg

def game_jeon(video_path, csv_path, output_folder, f_frame_count=92, l_frame_count=0, slow_duration_seconds=2, slow_rate=0.5):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c değişkeni kaydedilen pozisyon sayısını tutar
    c = 0
    
    # Dizin kontrolü ve oluşturma
    os.makedirs(output_folder, exist_ok=True)

    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'BLUE_ADD_GAME_JEON') | (df['matchLogItemType'] == 'RED_ADD_GAME_JEON')].iterrows():
        game_jeon_system_time = row['systemTime']

        # İlk önceki TIMEOUT değerini bul
        timeout_time = None
        for i in range(index - 1, -1, -1):
            if df.loc[i, 'matchLogItemType'] == 'TIMEOUT':
                timeout_time = df.loc[i, 'systemTime']
                break

        if timeout_time is None:
            print(f"Uyarı: {index}. indisteki GAME_JEON için TIMEOUT bulunamadı. Geçiliyor.")
            continue

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        timeout_video_time_seconds = (timeout_time - start_system_time) / 1000.0
        game_jeon_video_time_seconds = (game_jeon_system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        start_frame_number = int(timeout_video_time_seconds * fps)
        game_jeon_frame_number = int(game_jeon_video_time_seconds * fps)
        
        start_frame_number = max(0, min(frame_count - 1, start_frame_number))  # Frame sayısını kontrol etme
        game_jeon_frame_number = max(0, min(frame_count - 1, game_jeon_frame_number))  # Frame sayısını kontrol etme

        # Yavaşlatılacak frame sayısını hesapla
        slow_frame_count = int(slow_duration_seconds * fps)

        # Belirtilen sayıda önceki ve sonraki frame'leri al
        start_frame = max(0, start_frame_number - f_frame_count)
        end_frame = min(frame_count - 1, game_jeon_frame_number + l_frame_count)
        c += 1

        # Videoyu oku ve yeni videoya yaz
        output_video = []

        try:
            for i in range(start_frame, start_frame_number - slow_frame_count):
                if i >= frame_count:
                    break
                frame = video.get_data(i)
                output_video.append(frame)
            
            for i in range(start_frame_number - slow_frame_count, start_frame_number):
                if i >= frame_count:
                    break
                frame = video.get_data(i)
                # Yavaşlatılmış frame'leri tekrar ekle
                output_video.extend([frame] * int(1 / slow_rate))
            
            for i in range(start_frame_number, end_frame + 1):
                if i >= frame_count:
                    break
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c -= 1
            continue

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'game_jeon_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    
    print(f'Yan hakemlerin verdikleri Game-Jeonlar {output_folder} dosyasına kaydedildi. Toplam {c} pozisyon.')



In [ ]:
# Değişkenlerin tanımlanması
date = '02_13'
court_num = 3
cam_num = 'live'

video_konum = '/home/muhlabws4/AntalyaKodlari/downloads'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 13 FEBRUARY 2024 - 3 COURT.mp4'
video_path = f'{video_konum}/{video_name}'

csv_konum = '/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/matchlog'
csv_name = '02_13-320-334.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'


In [26]:
date='02_13'
court_num = 3
cam_num = 'live'

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='/home/muhlabws4/AntalyaKodlari/downloads'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = '11th TURKISH OPEN KYORUGI - G- 1 13 FEBRUARY 2024 - 3 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/3th_court/matchlog'
csv_name = '02_13-301-319_zaman.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/cam_{cam_num}/{court_num}th_court_{cam_num}_cam_{date}'

In [27]:

# Fonksiyon çağrısı
game_jeon(video_path, csv_path, f'{output_path}_game_jeon_temp2', f_frame_count=164, l_frame_count=31)

Yan hakemlerin verdikleri Game-Jeonlar /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_game_jeon_temp2 dosyasına kaydedildi. Toplam 86 pozisyon.
